# What about SCALING Factors and offsets - do they matter?

# Lake Isabel

- find the geojson
- determine the date range
- determine the band combinations
- find the best fire image
- consider doing again with Sentinel or HLS

https://www.usgs.gov/core-science-systems/nli/landsat/landsat-normalized-burn-ratio

![](https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/SR-NBR.JPG)

https://raw.githubusercontent.com/tonybutzer/notebook/e5bcf7eceaf276329cc7790a453b17600f773161/01-Everything-but-harmony/05-Science-Outreach-Evaporation-Etc/10-Sanath/01-Animation-Exploration/isa.gif

In [ ]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [ ]:
gifn = 'https://raw.githubusercontent.com/tonybutzer/notebook/e5bcf7eceaf276329cc7790a453b17600f773161/01-Everything-but-harmony/05-Science-Outreach-Evaporation-Etc/10-Sanath/01-Animation-Exploration/isa.gif'

# The below animation 

- was made for Sanath
- using Landsat 8  (Planet/AWS) and lite-stac - open data cube lite software - 2 years ago
- to many clouds - and used non-official data from Planet/AWS/USGS - Top-of-atmosphere
- required special custom code 

---

## Can we do better now - with cloud filtering etc?

### observation 2016-07-08 has a big fire

In [ ]:
display_gif(gifn)

In [ ]:
#!wget https://raw.githubusercontent.com/tonybutzer/notebook/e5bcf7eceaf276329cc7790a453b17600f773161/01-Everything-but-harmony/05-Science-Outreach-Evaporation-Etc/10-Sanath/01-Animation-Exploration/isabella-lake.geojson

In [ ]:
import sys
import os

import hvplot.xarray

In [ ]:
sys.path.append('./fakelib')
from nr_cube import NR
# this next line could be hiddent in __init__
os.environ["AWS_REQUEST_PAYER"] = "requester" 

In [ ]:
nr=NR()

In [ ]:
nr.measurements()

In [ ]:
! ls *.geojson

In [ ]:
min_cloud = 0
max_cloud = 30

date_min="2016-06-01"
date_max="2016-11-30"

aoi_geojson_file = './isa-big.geojson'

In [ ]:
# method to show our AOI

In [ ]:
nr.sat(aoi_geojson_file)

In [ ]:
my_measures = ['green', 'swir']
answer_stac, my_df = nr.search(aoi_geojson_file, date_min, date_max, min_cloud, max_cloud, my_measures, sort_date=True)

In [ ]:
my_df

In [ ]:
# need to add this to the library
#import pandas as pd
#my_df['datetime'] = pd.to_datetime(my_df['datetime'])

In [ ]:
df_pr = my_df.loc[my_df['landsat:wrs_row'] == '035']

In [ ]:
df_pr = df_pr.loc[df_pr['landsat:wrs_path'] == '041']

In [ ]:
#df_pr = df_pr[df_pr['red'].notna()]

In [ ]:
df_pr

In [ ]:
%%time
DS = nr.load(aoi_geojson_file, df_pr)

In [ ]:
my_df

In [ ]:
DS

In [ ]:
DS['red'][0].plot()

In [ ]:
DS['red'][0]

In [ ]:
DS['red'][1].plot()

In [ ]:
! ls fakelib

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
import skimage.exposure as ske

def juice(ary):
    ary_equalize_corrected = ske.equalize_hist(ary)
    return ary_equalize_corrected
    

In [ ]:
# FAIL DS['red'][1].values = juice(DS['red'][1].values)

In [ ]:
DS['red'][1].plot()

In [ ]:
ary = juice(DS['red'][1].values)

In [ ]:
ary

In [ ]:
DS['red'][1].values

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.plot(ary)
plt.show()

In [ ]:
aryb = DS['red'][1].values

plt.plot(aryb)
plt.show()

In [ ]:
plt.imshow(ary, cmap='hot')
plt.show()

In [ ]:
plt.imshow(aryb, cmap='hot')
plt.show()

In [ ]:
ds_jul = DS.sel(time='2016-07-08', method='nearest')

In [ ]:
ds_jul

|                                       |       | 
|---------------------------------------|-------| 
| Band Combinations for Landsat 8       |       | 
| Composite Name     Bands              |       | 
| Natural Color                         | 4 3 2 | 
| False Color (urban)                   | 7 6 4 | 
| Color Infrared (vegetation)           | 5 4 3 | 
| Agriculture                           | 6 5 2 | 
| Healthy Vegetation                    | 5 6 2 | 
| Land/Water                            | 5 6 4 | 
| Natural With Atmospheric Removal      | 7 5 3 | 
| Shortwave Infrared                    | 7 5 4 | 
| Vegetation Analysis                   | 6 5 4 | 


---
---
|                                       |       | 
|---------------------------------------|-------| 
| Healthy Vegetation                    | 5 6 2 | 
| Vegetation Analysis                   | 6 5 4 | 


In [ ]:
nr.measurements().T

In [ ]:
import numpy as np
import holoviews as hv


r = normalize(ds_jul['swir1'].values)
g = normalize(ds_jul['nir'].values)
b = normalize(ds_jul['blue'].values)

my_veggie_rgb = np.dstack([r,g,b])

# fail but not sure why hv.RGB(my_veggie_rgb).options(aspect='equal',  frame_width=500)

In [ ]:
b.mean()

In [ ]:
ds_jul['nir'].plot()

In [ ]:
ds_jul['swir1'].plot()

In [ ]:
ds_jul['blue'].plot()

In [ ]:
r.shape

In [ ]:
g.shape

In [ ]:
b.shape

In [ ]:
r = normalize(DS['red'].isel(time=(1)).values)
g = normalize(DS['green'].isel(time=(1)).values)
b = normalize(DS['blue'].isel(time=(1)).values)


In [ ]:
my_rgb = np.dstack([r,g,b])

In [ ]:
hv.RGB(my_rgb).options(aspect='equal',  frame_width=500)

In [ ]:
import skimage
rgb_equalize_corrected = skimage.exposure.equalize_hist(my_rgb)

In [ ]:
hv.RGB(rgb_equalize_corrected).options(aspect='equal',  frame_width=500)

In [ ]:
r = normalize(DS['swir1'].isel(time=(1)).values)
g = normalize(DS['nir'].isel(time=(1)).values)
b = normalize(DS['blue'].isel(time=(1)).values)


In [ ]:
my_rgb = np.dstack([r,g,b])

In [ ]:
hv.RGB(my_rgb).options(aspect='equal',  frame_width=500)

In [ ]:
DS.time

In [ ]:
r = normalize(DS['swir1'].isel(time=(2)).values)
g = normalize(DS['nir'].isel(time=(2)).values)
b = normalize(DS['blue'].isel(time=(2)).values)

In [ ]:
my_rgb = np.dstack([r,g,b])

In [ ]:
hv.RGB(my_rgb).options(aspect='equal',  frame_width=500)

In [ ]:
ds_u = DS.isel(time=(2))

In [ ]:
ds_u

In [ ]:
r = normalize(DS['swir1'].isel(time=(1)).values)
g = normalize(DS['nir'].isel(time=(1)).values)
b = normalize(DS['blue'].isel(time=(1)).values)

In [ ]:
DS['blue'].isel(time=(1)).time

In [ ]:
my_rgb = np.dstack([r,g,b])

In [ ]:
hv.RGB(my_rgb).options(aspect='equal',  frame_width=500)

https://www.usgs.gov/core-science-systems/nli/landsat/landsat-burned-area?qt-science_support_page_related_con=0#qt-science_support_page_related_con

# 5 4 3 - nir,blue, green

In [ ]:
r = normalize(DS['nir'].isel(time=(2)).values)
g = normalize(DS['blue'].isel(time=(2)).values)
b = normalize(DS['green'].isel(time=(2)).values)
my_rgb = np.dstack([r,g,b])
hv.RGB(my_rgb).options(aspect='equal',  frame_width=500)

In [ ]:
r = normalize(DS['nir'].isel(time=(1)).values)
g = normalize(DS['blue'].isel(time=(1)).values)
b = normalize(DS['green'].isel(time=(1)).values)
my_rgb1 = np.dstack([r,g,b])
hv.RGB(my_rgb1).options(aspect='equal',  frame_width=500)

In [ ]:
j_rgb = skimage.exposure.equalize_hist(my_rgb)

In [ ]:
j_rgb1 = skimage.exposure.equalize_hist(my_rgb1)

In [ ]:
hv.RGB(j_rgb1).options(aspect='equal',  frame_width=500)

In [ ]:
hv.RGB(j_rgb).options(aspect='equal',  frame_width=500)

![](https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/BurnedArea2.jpg)

In [ ]:
from nb_animate import nb_animated_timeseries

home = os.getenv('HOME')
gifn='isabella_animation_swn.gif'
nb_animated_timeseries(DS,output_path=gifn,bands = ['swir1', 'nir', 'green'], time_dim='time',  
      percentile_stretch=(.2,.85), interval=1200)

In [ ]:
gifn

In [ ]:

display_gif(gifn)

In [ ]:

home = os.getenv('HOME')
gifn='isabella_animation_543.gif'
nb_animated_timeseries(DS,output_path=gifn,bands = ['nir', 'blue', 'green'], time_dim='time',  
      percentile_stretch=(.2,.85), interval=1200)

In [ ]:
display_gif(gifn)

In [ ]:
DS

In [ ]:
from ani_easy import ani_easy
ani1 = ani_easy(DS['swir1'], cmap='hot')

In [ ]:
from IPython.display import HTML, display
HTML(ani1.to_jshtml())

In [ ]:
da_nbr = (DS['nir'] - DS['swir2']) / (DS['nir'] + DS['swir2'])
ds_nbr = da_nbr.to_dataset(name='nbr')

In [ ]:

ani2 = ani_easy(ds_nbr['nbr'], cmap='tab10')

In [ ]:
HTML(ani2.to_jshtml())